In [83]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

# Importing the dataset
dataset = pd.read_csv(r'C:/Users/4nm20/Desktop/Alternate Approach/new_normalized_output.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
num_features = X.shape[1]
print("Number of features:",num_features)
print(X)

Number of features: 4
[[0.00000000e+00 6.01902098e-05 1.00000000e+00 3.12591682e-01]
 [2.26449275e-04 6.95178276e-05 5.48676711e-01 3.14064430e-01]
 [4.52898551e-04 5.98924965e-05 3.91313184e-01 3.18715504e-01]
 ...
 [9.99547101e-01 3.81637043e-05 6.91692476e-02 7.13452354e-04]
 [9.99773551e-01 8.24844739e-05 6.91438305e-02 6.58572104e-04]
 [1.00000000e+00 1.27110904e-04 6.91438305e-02 0.00000000e+00]]


In [84]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[0 0 0 ... 2 2 2]


In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

#print(X_train, y_train)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [86]:
time_sequence_length = 4
X_train_rnn = []
y_train_rnn = []

for i in range(time_sequence_length, len(X_train)):
    X_train_rnn.append(X_train[i - time_sequence_length:i, :])
    y_train_rnn.append(y_train[i])

X_train_rnn, y_train_rnn = np.array(X_train_rnn), np.array(y_train_rnn)

In [87]:
time_sequence_length = 4
X_test_rnn = []
y_test_rnn = []

for i in range(time_sequence_length, len(X_test)):
    X_test_rnn.append(X_test[i - time_sequence_length:i, :])
    y_test_rnn.append(y_test[i])

X_test_rnn, y_test_rnn = np.array(X_test_rnn), np.array(y_test_rnn)
print(y_train_rnn)

[1 2 1 ... 2 1 0]


In [92]:
model = Sequential()
model.add(SimpleRNN(units=128, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
#model.add(SimpleRNN(units=128, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))



In [93]:
from keras.utils import to_categorical

# Assuming your original labels are in y_train_rnn
y_train_rnn_encoded = to_categorical(y_train_rnn, num_classes=3)
display(y_train_rnn_encoded)
print(y_train)


"""# Number of unique values in the original array
num_classes = len(np.unique(y_train_rnn))
print(num_classes)

# Create a one-hot encoding of the original array
one_hot_encoding = np.eye(num_classes)[y_train]

# Print the one-hot encoding
print(one_hot_encoding)"""

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

[1 2 2 ... 2 1 0]


'# Number of unique values in the original array\nnum_classes = len(np.unique(y_train_rnn))\nprint(num_classes)\n\n# Create a one-hot encoding of the original array\none_hot_encoding = np.eye(num_classes)[y_train]\n\n# Print the one-hot encoding\nprint(one_hot_encoding)'

In [96]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_rnn, y_train_rnn_encoded, batch_size=16, epochs=60)

Epoch 1/60
643/643 [==============================] - 5s 5ms/step - loss: 1.0800 - accuracy: 0.3951
Epoch 2/60
643/643 [==============================] - 3s 4ms/step - loss: 1.0823 - accuracy: 0.3975
Epoch 3/60
643/643 [==============================] - 3s 4ms/step - loss: 1.0791 - accuracy: 0.4015
Epoch 4/60
643/643 [==============================] - 3s 4ms/step - loss: 1.0796 - accuracy: 0.4027
Epoch 5/60
643/643 [==============================] - 3s 4ms/step - loss: 1.0812 - accuracy: 0.4016
Epoch 6/60
643/643 [==============================] - 3s 5ms/step - loss: 1.0811 - accuracy: 0.3953
Epoch 7/60
643/643 [==============================] - 3s 5ms/step - loss: 1.0780 - accuracy: 0.4055
Epoch 8/60
643/643 [==============================] - 3s 4ms/step - loss: 1.0768 - accuracy: 0.4025
Epoch 9/60
643/643 [==============================] - 3s 5ms/step - loss: 1.0808 - accuracy: 0.3963
Epoch 10/60
643/643 [==============================] - 3s 4ms/step - loss: 1.0791 - accuracy: 0.3949

In [97]:
y_pred = model.predict(X_test_rnn)
#print(y_pred)
predicted_class_index = np.argmax(y_pred)
predicted_class = predicted_class_index
print(predicted_class)

81/81 [==============================] - 0s 1ms/step
494


In [26]:
sample_data = np.array([[0.9952445652173914,4.7027858242244615e-05,0.06919466468635627,0.12254955582596286]])
# Create a sequence of 10 identical data points (assuming all attributes have the same value)
sample_sequence = np.array([sample_data] * 4)
# Reshape the sample data to match the input shape
sample_sequence = sample_data.reshape(1, 4, 3)  # 1 sample, sequence length of 10, and 3 attributes
# Make predictions
predictions = model.predict(sample_sequence)
#print(predictions)

predicted_class_index = np.argmax(predictions)
predicted_class = predicted_class_index
print(predicted_class)

ValueError: cannot reshape array of size 4 into shape (1,4,3)

In [68]:
import numpy as np

sample_data = np.array([[0.9995471014492754,3.816370433047557e-05,0.0691692475684599,0.0007134523536799654]])

# Create a sequence of 10 identical data points (assuming all attributes have the same value)
sample_sequence = np.array([sample_data] * 10)  # Create a sequence of 10 data points

# Reshape the sample data to match the input shape
sample_sequence = sample_sequence.reshape(1, 10, 4)  # 1 sample, sequence length of 10, and 4 attributes

# Make predictions
predictions = model.predict(sample_sequence)

predicted_class_index = np.argmax(predictions)
predicted_class = predicted_class_index
print(predicted_class)


1/1 [==============================] - 0s 51ms/step
1
